### 통계적 분석

In [45]:
### 라이브러리
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split

from scipy import stats
from scipy.stats import shapiro

import statsmodels.api as sm
from statsmodels.tsa.stattools import grangercausalitytests
from statsmodels.stats.stattools import jarque_bera
from statsmodels.stats.outliers_influence import variance_inflation_factor

from sklearn.preprocessing import StandardScaler

import datetime

### 환경설정
np.random.seed(4)


### 함수 정의
def preprocess_transaction_flag(df):
    df['transaction_flag'] = df['transaction_count'].apply(lambda x: 1 if x > 0 else 0)
    return df


def trim_date(df, start, end):
    ### df date 형식 맞춰주기
    df['date'] = df['date'].astype(str)
    if len(df['date'][0]) == 8:
        df['date'] = df['date'].apply(lambda x: x[0:4]+'-'+x[4:6]+'-'+x[6:])


    return df[(df['date']>=start) & (df['date']<=end)]

### 데이터 세팅

In [ ]:
### 학습용 데이터 읽기
test_df = pd.read_csv("data/test/test_df.csv")
test_df.set_index('date', inplace=True)

### 이벤트용 데이터 읽기
event_df = pd.read_csv("./data/event/bitcoin_event_detail_no_explain.csv")
event_df.index = event_df['Date'].astype(str).apply(lambda x: x[0:4]+'-'+x[4:6]+'-'+x[6:])
# event_df = event_df[event_df['predict']==1]


target_event_receive_df = pd.read_csv("./data/target/event_2_day_transactions_over_136_threshold_1000_receive.csv")
target_event_receive_df = preprocess_transaction_flag(target_event_receive_df)
target_event_receive_df.columns = ['date', 'transaction_count', 'transaction_amount', 'transaction_flag']

target_event_receive_df = trim_date(target_event_receive_df, (test_df.index)[0], (test_df.index)[-1])



### Bitcoin Event Flag 추가 / 긍정, 부정, 애매 이벤트 필터링
# pos, neg, neu, all
event_sentiment = 'neg'
if event_sentiment == 'all':
    pass
elif event_sentiment == 'pos':
    event_df = event_df[event_df['classification']==1]
elif event_sentiment == 'neg':
    event_df = event_df[event_df['classification']==0]
elif event_sentiment == 'neu':
    event_df = event_df[event_df['classification']==2]
event_df['classification'] = 1
event_df = event_df['classification']
event_df = event_df[~event_df.index.duplicated(keep='first')]

test_df['event_flag'] = event_df
test_df['event_flag'] = test_df['event_flag'].fillna(0)
event_flag_copy = test_df['event_flag'].values

### 분석 데이터 선택
target_df = target_event_receive_df.copy().set_index('date')

### test_df에 target_df 데이터 적용
test_df['transaction_count'] = target_df['transaction_count']
test_df['transaction_amount'] = target_df['transaction_amount']
test_df['transaction_flag'] = target_df['transaction_flag']
test_df['transaction_amount_usd'] = test_df['transaction_amount'] * test_df['close']

### test_df의 target 선택
test_df['target'] = test_df['transaction_amount_usd']
test_df['target'] = np.where(test_df['target'] == 0, 1e-10, test_df['target'])

### target 관련 파생변수 생성
test_df['target_delta'] = test_df['target'].diff(1)
test_df['target_returns'] = test_df['target'].pct_change(1)

### VPIN 관련 변수 추가
vpin = pd.read_csv('./data/vpin/vpin.csv')
vpin['ma_10'] = vpin['vpin'].rolling(10).mean()
vpin['ma_10'] = vpin['ma_10'].fillna(vpin['ma_10'].mean())
vpin = vpin.set_index('date')

test_df['vpin'] = vpin['vpin']
test_df['vpin_ma_10'] = vpin['ma_10']

##### 전처리 관련
test_df = test_df.replace([np.inf, -np.inf], np.nan)
test_df = test_df.fillna(test_df.mean())

C:\Users\slaye\AppData\Local\Temp\ipykernel_40084\4114716843.py:53: FutureWarning: The default fill_method='pad' in Series.pct_change is deprecated and will be removed in a future version. Either fill in any non-leading NA values prior to calling pct_change or specify 'fill_method=None' to not fill NA values.
  test_df['target_returns'] = test_df['target'].pct_change(1)


In [47]:
test_df[['returns', 'transaction_amount', 'target', 'target_delta', 'target_returns', 'vpin']].describe(percentiles=[.01, .25, .5, .75, .99]).T

,count,mean,std,min,1%,25%,50%,75%,99%,max
returns,2830.0,1.612090e-03,3.607990e-02,-3.950485e-01,-9.999244e-02,-1.227509e-02,0.000000e+00,1.529362e-02,1.063411e-01,2.250139e-01
transaction_amount,2830.0,1.851405e+05,9.541025e+05,0.000000e+00,0.000000e+00,0.000000e+00,1.000000e+04,1.285258e+05,2.228565e+06,2.491523e+07
target,2830.0,1.823643e+09,5.089270e+09,1.000000e-10,1.000000e-10,1.000000e-10,2.242831e+08,1.823643e+09,1.598677e+10,1.066540e+11
target_delta,2830.0,-1.511529e+06,4.740623e+09,-5.895144e+10,-9.631444e+09,-1.964618e+08,0.000000e+00,8.958915e+07,1.046060e+10,6.482819e+10
target_returns,2830.0,2.304158e+18,1.682938e+19,-1.000000e+00,-1.000000e+00,-2.498244e-01,0.000000e+00,2.406381e-01,5.823788e+19,5.895144e+20
vpin,2830.0,6.453470e-01,6.112185e-02,9.000000e-04,4.836492e-01,6.098612e-01,6.482219e-01,6.859938e-01,7.729164e-01,8.470440e-01


### 기본적인 통계, 시계열 분석

In [48]:
stats_df = test_df.copy()

print(stats_df[['target', 'target_delta', 'target_returns']].describe())

stats_df.index = pd.to_datetime(stats_df.index)

# 연도별 event_flag 개수 확인
event_count_per_year = stats_df.groupby(stats_df.index.year)['event_flag'].sum()

# 결과 출력
print(event_count_per_year)


             target  target_delta  target_returns
count  2.830000e+03  2.830000e+03    2.830000e+03
mean   1.823643e+09 -1.511529e+06    2.304158e+18
std    5.089270e+09  4.740623e+09    1.682938e+19
min    1.000000e-10 -5.895144e+10   -1.000000e+00
25%    1.000000e-10 -1.964618e+08   -2.498244e-01
50%    2.242831e+08  0.000000e+00    0.000000e+00
75%    1.823643e+09  8.958915e+07    2.406381e-01
max    1.066540e+11  6.482819e+10    5.895144e+20
date
2017    2.0
2018    5.0
2019    1.0
2020    0.0
2021    5.0
2022    8.0
2023    1.0
2024    0.0
Name: event_flag, dtype: float64


In [49]:
lagged_event_df = test_df.copy()
lagged_event_df['transaction_flag'].value_counts()

transaction_flag
1    1572
0    1258
Name: count, dtype: int64

In [50]:
lagged_event_df = test_df.copy()
lagged_event_df['lagged_event_flag'] = lagged_event_df['event_flag'].shift(-1).fillna(0)
lagged_event_df[lagged_event_df['lagged_event_flag']==1]['transaction_flag'].value_counts()

transaction_flag
1    15
0     7
Name: count, dtype: int64

### VIF: Volume과 Target Delta 사이 설명력 중복 확인

In [51]:

# Target Delta를 적절히 Lagging
test_df['lag_target_delta'] = test_df['target_delta'].shift(1)

# Independent, Dependent 분리
y = test_df['returns'].shift(-3).fillna(test_df['returns'].mean())
X = test_df[['lag_target_delta', 'target', 'volume']].fillna(test_df.mean())


X['returns'] = test_df['returns'].shift(-3).fillna(test_df['returns'].mean())

## train, test 데이터 분리
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.6, shuffle=False)

start = '2023-01-01'
end = '2024-12-31'
X_test = X[(X.index >= start)&(X.index <= end)]
y_test = y[(y.index >= start)&(y.index <= end)]


print(X_test[['returns', 'lag_target_delta', 'target', 'volume']].corr())  # 상관관계 확인

# VIF 계산
X = X[['lag_target_delta', 'target', 'volume']]  # OLS 모델에서 사용한 독립변수들
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4, shuffle=False)
vif_data = pd.DataFrame()
vif_data["Feature"] = X_test.columns
vif_data["VIF"] = [variance_inflation_factor(X_test.values, i) for i in range(X_test.shape[1])]

print(vif_data)

                   returns  lag_target_delta    target    volume
returns           1.000000          0.028948 -0.103140  0.101290
lag_target_delta  0.028948          1.000000  0.328641  0.011174
target           -0.103140          0.328641  1.000000 -0.053588
volume            0.101290          0.011174 -0.053588  1.000000
            Feature       VIF
0  lag_target_delta  1.058397
1            target  1.168842
2            volume  1.108058


### Event 이후 Statistics

In [52]:
event_df

Date
2016-08-03    1
2017-12-07    1
2017-12-19    1
2018-01-26    1
2018-05-01    1
2018-06-01    1
2018-07-09    1
2018-09-01    1
2019-05-07    1
2021-08-01    1
2021-10-28    1
2021-12-02    1
2021-12-06    1
2021-12-12    1
2022-01-27    1
2022-03-01    1
2022-05-09    1
2022-09-20    1
2022-11-02    1
2022-11-07    1
2022-11-10    1
2022-11-11    1
2023-09-25    1
Name: classification, dtype: int64

In [53]:
event_df = event_df.to_frame()
event_df

,classification
Date,
2016-08-03,1
2017-12-07,1
2017-12-19,1
2018-01-26,1
2018-05-01,1
2018-06-01,1
2018-07-09,1
2018-09-01,1
2019-05-07,1


In [54]:

# 수익률 계산 함수
def calculate_returns(test_df, event_df, days):
    returns = {}
    event_df.index = pd.to_datetime(event_df.index).date
    for date in event_df.index:
        future_date = (date + pd.Timedelta(days=days)).strftime('%Y-%m-%d')
        date = date.strftime('%Y-%m-%d')
        if future_date in test_df.index:
            ret = (test_df.loc[future_date, 'close'] - test_df.loc[date, 'close']) / test_df.loc[date, 'close']
            returns[date] = ret
        else:
            returns[date] = None  # 미래 데이터가 없는 경우
    return pd.Series(returns)

# 각 기간의 수익률 계산
event_df['Return_7d'] = list(calculate_returns(test_df, event_df, 1))
event_df['Return_7d'].mean()


# event_df['Return_30d'] = calculate_returns(test_df, event_df, 30)
# event_df['Return_90d'] = calculate_returns(test_df, event_df, 90)

-0.00828780187938726

In [55]:
event_df.Return_7d.mean()

-0.00828780187938726